# **Cuaderno de ETL: De Datos Abiertos a un Modelo de Estrella**

**Objetivo:** Tomar un conjunto de datos públicos sobre estadísticas de educación en Colombia y transformarlo en un modelo dimensional de estrella, listo para ser analizado con herramientas de Business Intelligence.

**Fuente de Datos:** [MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA](https://www.datos.gov.co/Educaci-n/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR-B-SICA/nudc-7mev/about_data)

**Nuestro Modelo de Estrella a Construir:**

* **Tabla de Hechos (Fact_Matriculas):**
    * `id_tiempo` (FK)
    * `id_geografia` (FK)
    * `total_matriculados` (Métrica)
    ...

* **Tablas de Dimensiones:**
    * `Dim_Tiempo` (año)
    * `Dim_Geografia` (departamento, municipio)

¡Manos a la obra!

In [3]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [4]:
df_raw

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14580,2011,5036,Angelópolis,5,Antioquia,3758,Antioquia (ETC),1707,78.85,78.9,...,3.61,9.5,7.32,0.71,0,0.7,1.08,0,19.57,100
14581,2011,5034,Andes,5,Antioquia,3758,Antioquia (ETC),10244,84.45,84.5,...,0.58,0.04,2.69,5.41,0.73,5.53,6.9,4.11,24.43,93.44
14582,2011,5031,Amalfi,5,Antioquia,3758,Antioquia (ETC),5552,97.71,97.7,...,0,0,0,NaN,0.83,NaN,9.93,4.47,20.01,53.45
14583,2011,5030,Amagá,5,Antioquia,3758,Antioquia (ETC),6631,78.65,78.7,...,6.73,14.46,7.45,0.42,0,0.24,0.91,0,25.05,83.33


In [5]:
# Vemos las columnas disponibles
print(df_raw.columns)

# Convertimos nombres a minúsculas por consistencia
df_raw.columns = df_raw.columns.str.lower()

# Vemos valores únicos para columnas clave
print("Años únicos:", df_raw['a_o'].unique())
print("Departamentos únicos:", df_raw['departamento'].unique())
print("Municipios únicos:", df_raw['municipio'].nunique())


Index(['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento',
       'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16',
       'tasa_matriculaci_n_5_16', 'cobertura_neta',
       'cobertura_neta_transici_n', 'cobertura_neta_primaria',
       'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta',
       'cobertura_bruta_transici_n', 'cobertura_bruta_primaria',
       'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n',
       'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria',
       'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n',
       'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media',
       'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria',
       'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia',
       'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
       'repitencia_media', 'tama_o_promedio_de_grupo',
       'sedes_conectadas_a_internet'],
   

In [6]:
dim_tiempo = df_raw[['a_o']].drop_duplicates().copy()
dim_tiempo['id_tiempo'] = dim_tiempo.reset_index().index + 1  # ID autoincremental
dim_tiempo.rename(columns={'a_o': 'anio'}, inplace=True)


In [7]:
dim_geografia = df_raw[['departamento', 'municipio']].drop_duplicates().copy()
dim_geografia['id_geografia'] = dim_geografia.reset_index().index + 1


# primera forma normal

In [ ]:

for col in df_raw.columns:
    if df_raw[col].apply(lambda x: isinstance(x, list)).any():
        print(f"La columna {col} contiene listas y debe ser normalizada.")

# Revisamos campos mal formateados
df_raw['departamento'] = df_raw['departamento'].str.strip().str.upper()
df_raw['municipio'] = df_raw['municipio'].str.strip().str.upper()


#  Segunda Forma Normal (2FN): Dependencia Total de la Clave


In [9]:
# Creamos una Dim_Geografia sin redundancias
dim_geografia = df_raw[['departamento', 'municipio']].drop_duplicates().copy()
dim_geografia['id_geografia'] = dim_geografia.reset_index().index + 1


# Tercera Forma Normal (3FN): Sin Dependencias Transitivas

In [11]:
print(df_raw.columns.tolist())


['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento', 'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_neta_transici_n', 'cobertura_neta_primaria', 'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n', 'cobertura_bruta_primaria', 'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria', 'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tama_o_promedio_de_grupo', 'sedes_conectadas_a_internet']


In [12]:
df_raw['matriculados_estimados'] = (
    pd.to_numeric(df_raw['tasa_matriculaci_n_5_16'], errors='coerce') / 100
) * pd.to_numeric(df_raw['poblaci_n_5_16'], errors='coerce')


In [ ]:
df_raw.rename(columns={
    'a_o': 'anio',
    'c_digo_municipio': 'cod_municipio',
    'municipio': 'municipio',
    'c_digo_departamento': 'cod_departamento',
    'departamento': 'departamento',
    'poblaci_n_5_16': 'poblacion_5_16',
    'tasa_matriculaci_n_5_16': 'tasa_matriculacion',
    'matriculados_estimados': 'total_matriculados'  
}, inplace=True)


In [14]:
# Renombramos columnas para facilitar trabajo
df_raw.rename(columns={
    'a_o': 'anio',
    'c_digo_municipio': 'cod_municipio',
    'municipio': 'municipio',
    'c_digo_departamento': 'cod_departamento',
    'departamento': 'departamento',
    'poblaci_n_5_16': 'poblacion_5_16',
    'tasa_matriculaci_n_5_16': 'tasa_matriculacion'
}, inplace=True)

# Calculamos estudiantes matriculados estimados
df_raw['poblacion_5_16'] = pd.to_numeric(df_raw['poblacion_5_16'], errors='coerce')
df_raw['tasa_matriculacion'] = pd.to_numeric(df_raw['tasa_matriculacion'], errors='coerce')

df_raw['total_matriculados'] = (df_raw['tasa_matriculacion'] / 100) * df_raw['poblacion_5_16']

# Eliminamos filas con valores clave nulos
df_raw = df_raw.dropna(subset=['anio', 'departamento', 'municipio', 'total_matriculados'])

# Estandarizamos nombres
df_raw['departamento'] = df_raw['departamento'].str.upper().str.strip()
df_raw['municipio'] = df_raw['municipio'].str.upper().str.strip()

print("✅ Columnas limpias y métrica lista:")
display(df_raw[['anio', 'departamento', 'municipio', 'total_matriculados']].head())


✅ Columnas limpias y métrica lista:


C:\Users\juanF\AppData\Local\Temp\ipykernel_25120\3789554957.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw['departamento'] = df_raw['departamento'].str.upper().str.strip()
C:\Users\juanF\AppData\Local\Temp\ipykernel_25120\3789554957.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw['municipio'] = df_raw['municipio'].str.upper().str.strip()


,anio,departamento,municipio,total_matriculados
0,2023,ANTIOQUIA,ABRIAQUÍ,314.9786
1,2023,GUAVIARE,EL RETORNO,2364.1226
2,2023,GUAVIARE,MIRAFLORES,654.9528
3,2023,VAUPÉS,MITÚ,6544.3602
4,2023,VAUPÉS,CARURU,629.9640


# Dimensión Dim_Tiempo (solo año)

In [15]:
# Creamos Dim_Tiempo sin duplicados
dim_tiempo = df_raw[['anio']].drop_duplicates().copy()

# Asignamos un ID subrogado
dim_tiempo['id_tiempo'] = dim_tiempo.reset_index().index + 1

# Reordenamos columnas
dim_tiempo = dim_tiempo[['id_tiempo', 'anio']]

print("✅ Dim_Tiempo lista:")
display(dim_tiempo.head())


✅ Dim_Tiempo lista:


,id_tiempo,anio
0,1,2023
1121,2,2022
2242,3,2021
3364,4,2020
4486,5,2019


# Dimensión Dim_Geografia (departamento y municipio)

In [16]:
# Creamos Dim_Geografia
dim_geografia = df_raw[['departamento', 'municipio']].drop_duplicates().copy()

# Asignamos un ID subrogado
dim_geografia['id_geografia'] = dim_geografia.reset_index().index + 1

# Reordenamos columnas
dim_geografia = dim_geografia[['id_geografia', 'departamento', 'municipio']]

print("✅ Dim_Geografia lista:")
display(dim_geografia.head())


✅ Dim_Geografia lista:


,id_geografia,departamento,municipio
0,1,ANTIOQUIA,ABRIAQUÍ
1,2,GUAVIARE,EL RETORNO
2,3,GUAVIARE,MIRAFLORES
3,4,VAUPÉS,MITÚ
4,5,VAUPÉS,CARURU


        Dim_Tiempo           Dim_Geografia
            ↑                      ↑
            |                      |
         id_tiempo          id_geografia
               \              /
                \            /
                Fact_Matriculas
                   (total_matriculados)


In [17]:
# URL del API Socrata
divipola_url = "https://www.datos.gov.co/resource/gdxc-w37w.json?$limit=50000"

# Cargar los datos del DIVIPOLA
try:
    response_divipola = requests.get(divipola_url)
    response_divipola.raise_for_status()
    data_divipola = response_divipola.json()
    df_divipola = pd.DataFrame(data_divipola)

    print(f"✅ DIVIPOLA cargado con {len(df_divipola)} filas.")
    display(df_divipola.head())

except Exception as e:
    print(f"❌ Error al descargar DIVIPOLA: {e}")
    df_divipola = pd.DataFrame()


✅ DIVIPOLA cargado con 1122 filas.


,cod_dpto,dpto,cod_mpio,nom_mpio,tipo_municipio,longitud,latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,05,ANTIOQUIA,05002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,05,ANTIOQUIA,05004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"
3,05,ANTIOQUIA,05021,ALEJANDRÍA,Municipio,"-75,141346","6,376061"
4,05,ANTIOQUIA,05030,AMAGÁ,Municipio,"-75,702188","6,038708"


In [19]:
print(df_divipola.columns.tolist())


['cod_dpto', 'dpto', 'cod_mpio', 'nom_mpio', 'tipo_municipio', 'longitud', 'latitud']


In [20]:
# Renombramos columnas para que coincidan con Dim_Geografia
df_divipola = df_divipola.rename(columns={
    'cod_dpto': 'cod_departamento',
    'dpto': 'departamento_divipola',
    'cod_mpio': 'cod_municipio',
    'nom_mpio': 'municipio_divipola'
})

# Limpieza de texto para hacer match
df_divipola['departamento_divipola'] = df_divipola['departamento_divipola'].str.upper().str.strip()
df_divipola['municipio_divipola'] = df_divipola['municipio_divipola'].str.upper().str.strip()


In [21]:
# Hacemos la unión por nombre de departamento y municipio
dim_geografia_ext = dim_geografia.merge(
    df_divipola,
    left_on=['departamento', 'municipio'],
    right_on=['departamento_divipola', 'municipio_divipola'],
    how='left'
)

# Seleccionamos columnas finales
dim_geografia_ext = dim_geografia_ext[[
    'id_geografia',
    'cod_departamento',
    'departamento',
    'cod_municipio',
    'municipio'
]]

print("✅ Dim_Geografia extendida con DIVIPOLA:")
display(dim_geografia_ext.head())


✅ Dim_Geografia extendida con DIVIPOLA:


,id_geografia,cod_departamento,departamento,cod_municipio,municipio
0,1,05,ANTIOQUIA,05004,ABRIAQUÍ
1,2,95,GUAVIARE,95025,EL RETORNO
2,3,95,GUAVIARE,95200,MIRAFLORES
3,4,97,VAUPÉS,97001,MITÚ
4,5,NaN,VAUPÉS,NaN,CARURU


In [22]:
# Verifica que no haya duplicados en el ID geográfico
assert dim_geografia_ext['id_geografia'].is_unique, "⚠️ Hay IDs duplicados en Dim_Geografia"


In [23]:
# ¿Cuántas filas no lograron unirse con el DIVIPOLA?
sin_codigo = dim_geografia_ext[dim_geografia_ext['cod_municipio'].isnull()]
print(f"❗ Municipios sin código DANE: {len(sin_codigo)}")
display(sin_codigo)


❗ Municipios sin código DANE: 81


,id_geografia,cod_departamento,departamento,cod_municipio,municipio
4,5,NaN,VAUPÉS,NaN,CARURU
7,8,NaN,VAUPÉS,NaN,PAPUNAUA
19,20,NaN,GUAINÍA,NaN,BARRANCO MINAS
26,27,NaN,AMAZONAS,NaN,MIRITI - PARANÁ
40,41,NaN,PUTUMAYO,NaN,LEGUÍZAMO
...,...,...,...,...,...
1121,1122,NaN,NACIONAL,NaN,NACIONAL
1122,1123,NaN,ARCHIPIÉLAGO DE SAN ANDRÉS. PROVIDENCIA Y SANT...,NaN,PROVIDENCIA
1123,1124,NaN,ARCHIPIÉLAGO DE SAN ANDRÉS. PROVIDENCIA Y SANT...,NaN,SAN ANDRÉS
1124,1125,NaN,BOGOTÁ D.C.,NaN,BOGOTÁ D.C.


In [25]:
# Unimos df_raw con las dimensiones
df_temp = df_raw.merge(dim_tiempo, on='anio', how='left')
df_temp = df_temp.merge(dim_geografia_ext, on=['departamento', 'municipio'], how='left')

# Creamos la tabla de hechos con las columnas necesarias
fact_matriculas = df_temp[['id_tiempo', 'id_geografia', 'total_matriculados']].copy()

# Revisión rápida
print("✅ Fact_Matriculas creada:")
display(fact_matriculas.head())


✅ Fact_Matriculas creada:


,id_tiempo,id_geografia,total_matriculados
0,1,1,314.9786
1,1,2,2364.1226
2,1,3,654.9528
3,1,4,6544.3602
4,1,5,629.9640


In [31]:
# Unimos df_raw con dimensiones
df_temp = df_raw.merge(dim_tiempo, on='anio', how='left')
df_temp = df_temp.merge(dim_geografia_ext, on=['departamento', 'municipio'], how='left')

# Construimos tabla de hechos
fact_matriculas = df_temp[['id_tiempo', 'id_geografia', 'total_matriculados']].copy()

# Conversión de tipos y tratamiento de nulos
fact_matriculas['id_tiempo'] = fact_matriculas['id_tiempo'].astype(int)
fact_matriculas['id_geografia'] = fact_matriculas['id_geografia'].astype(int)
fact_matriculas['total_matriculados'] = fact_matriculas['total_matriculados'].fillna(0).astype(float)

print("✅ Tabla de hechos creada correctamente.")


✅ Tabla de hechos creada correctamente.


In [26]:
fact_matriculas['id_tiempo'] = fact_matriculas['id_tiempo'].astype(int)
fact_matriculas['id_geografia'] = fact_matriculas['id_geografia'].astype(int)
fact_matriculas['total_matriculados'] = fact_matriculas['total_matriculados'].fillna(0).astype(float)


## **3. Transformación (Transform)**

Esta es la fase más importante. Aquí limpiamos los datos crudos y los moldeamos para que encajen en nuestro modelo de estrella.

**Pasos:**
1.  **Limpieza y Preparación:** Convertiremos las columnas a los tipos de datos correctos y manejaremos valores faltantes. La columna `matricula` es nuestra métrica principal.
2.  **Creación de Dimensiones:** A partir del DataFrame limpio, crearemos una tabla (DataFrame) para cada dimensión, asegurándonos de que no tengan filas duplicadas y asignando una **llave subrogada** (un ID numérico único).
3.  **Creación de la Tabla de Hechos:** Construiremos la tabla de hechos, que contendrá nuestra métrica (`total_matriculados`) y las llaves foráneas que la conectan a cada dimensión.

In [29]:
import sqlite3
import os


# Conexión a la base de datos
conn = sqlite3.connect("../Datos/modelo_estrella_educacion.db")


In [30]:
cursor = conn.cursor()

# Guardamos las tablas
dim_tiempo.to_sql("Dim_Tiempo", conn, if_exists="replace", index=False)
dim_geografia_ext.to_sql("Dim_Geografia", conn, if_exists="replace", index=False)
fact_matriculas.to_sql("Fact_Matriculas", conn, if_exists="replace", index=False)

print("✅ Tablas guardadas exitosamente en SQLite.")


✅ Tablas guardadas exitosamente en SQLite.


In [32]:
poblacion_escolar = df_raw[['anio', 'departamento', 'municipio', 'poblacion_5_16']].drop_duplicates()

# Unimos con las dimensiones para obtener ID's
poblacion_escolar = poblacion_escolar.merge(dim_tiempo, on='anio', how='left')
poblacion_escolar = poblacion_escolar.merge(dim_geografia_ext, on=['departamento', 'municipio'], how='left')

# Dejamos solo columnas clave
poblacion_escolar = poblacion_escolar[['id_tiempo', 'id_geografia', 'poblacion_5_16']]

# Guardamos en SQLite
poblacion_escolar.to_sql("Poblacion_Escolar", conn, if_exists="replace", index=False)

print("✅ Tabla Poblacion_Escolar cargada en SQLite.")


✅ Tabla Poblacion_Escolar cargada en SQLite.


## Preguntas

1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

2. ¿Cómo compararía el rendimiento educativo por municipios?

3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

In [34]:
def ejecutar_sql(query, conexion=conn):
    """
    Ejecuta una consulta SQL y devuelve el resultado como un DataFrame.
    """
    df = pd.read_sql_query(query, conexion)
    display(df)


In [37]:
query = """
SELECT 
    g.departamento,
    g.municipio,
    t.anio,
    SUM(f.total_matriculados) AS total_matriculados,
    SUM(p.poblacion_5_16) AS poblacion_5_16,
    ROUND(100.0 * SUM(f.total_matriculados) / SUM(p.poblacion_5_16), 2) || '%' AS porcentaje_escolaridad
FROM Fact_Matriculas f
JOIN Poblacion_Escolar p ON f.id_tiempo = p.id_tiempo AND f.id_geografia = p.id_geografia
JOIN Dim_Tiempo t ON f.id_tiempo = t.id_tiempo
JOIN Dim_Geografia g ON f.id_geografia = g.id_geografia
WHERE p.poblacion_5_16 > 0
GROUP BY g.departamento, g.municipio, t.anio
HAVING SUM(p.poblacion_5_16) > 0
ORDER BY porcentaje_escolaridad DESC
LIMIT 20;
"""

ejecutar_sql(query)


,departamento,municipio,anio,total_matriculados,poblacion_5_16,porcentaje_escolaridad
0,BOLÍVAR,ACHÍ,2021,5.935813,5.937,99.98%
1,PUTUMAYO,VILLAGARZÓN,2015,5472.905200,5474.000,99.98%
2,ARAUCA,ARAUCA,2021,20.446864,20.453,99.97%
3,BOYACÁ,PAIPA,2011,7033.889200,7036.000,99.97%
4,MAGDALENA,SAN SEBASTIÁN DE BUENAVISTA,2023,5120.950800,5123.000,99.96%
5,ANTIOQUIA,LIBORINA,2018,1997.001000,1998.000,99.95%
6,ATLÁNTICO,PALMAR DE VARELA,2014,5593.202000,5596.000,99.95%
7,CUNDINAMARCA,TOCANCIPÁ,2015,7983.207200,7988.000,99.94%
8,META,EL CASTILLO,2021,1.704976,1.706,99.94%
9,SANTANDER,RIONEGRO,2019,5415.206700,5419.000,99.93%


In [38]:
# Seleccionamos columnas relevantes
rendimiento = df_raw[['anio', 'departamento', 'municipio',
                      'aprobaci_n', 'reprobaci_n', 'repitencia']].copy()

# Renombramos columnas
rendimiento.rename(columns={
    'aprobaci_n': 'tasa_aprobacion',
    'reprobaci_n': 'tasa_reprobacion',
    'repitencia': 'tasa_repitencia'
}, inplace=True)

# Unimos con las dimensiones
rendimiento = rendimiento.merge(dim_tiempo, on='anio', how='left')
rendimiento = rendimiento.merge(dim_geografia_ext, on=['departamento', 'municipio'], how='left')

# Filtramos columnas finales
rendimiento = rendimiento[['id_tiempo', 'id_geografia', 'tasa_aprobacion', 'tasa_reprobacion', 'tasa_repitencia']]

# Convertimos a numérico
for col in ['tasa_aprobacion', 'tasa_reprobacion', 'tasa_repitencia']:
    rendimiento[col] = pd.to_numeric(rendimiento[col], errors='coerce')

# Guardamos en SQLite
rendimiento.to_sql("Rendimiento_Educativo", conn, if_exists="replace", index=False)

print("✅ Tabla Rendimiento_Educativo cargada.")


✅ Tabla Rendimiento_Educativo cargada.


In [40]:
query = """
SELECT 
    g.departamento,
    g.municipio,
    t.anio,
    ROUND(AVG(r.tasa_aprobacion), 3) || '%' AS promedio_aprobacion,
    ROUND(AVG(r.tasa_reprobacion), 3) || '%' AS promedio_reprobacion,
    ROUND(AVG(r.tasa_repitencia), 3) || '%' AS promedio_repitencia
FROM Rendimiento_Educativo r
JOIN Dim_Tiempo t ON r.id_tiempo = t.id_tiempo
JOIN Dim_Geografia g ON r.id_geografia = g.id_geografia
WHERE r.tasa_aprobacion IS NOT NULL
GROUP BY g.departamento, g.municipio, t.anio
ORDER BY promedio_aprobacion DESC
LIMIT 20;
"""

ejecutar_sql(query)



,departamento,municipio,anio,promedio_aprobacion,promedio_reprobacion,promedio_repitencia
0,NARIÑO,LA TOLA,2016,99.95%,0.0%,0.19%
1,NARIÑO,MOSQUERA,2016,99.95%,0.0%,1.28%
2,NARIÑO,CONSACA,2017,99.94%,0.0%,0.94%
3,NARIÑO,MAGÜI,2012,99.94%,0.0%,7.38%
4,META,BARRANCA DE UPÍA,2014,99.93%,0.0%,0.0%
5,NARIÑO,FRANCISCO PIZARRO,2015,99.91%,0.09%,0.0%
6,NARIÑO,LA TOLA,2015,99.91%,0.0%,0.05%
7,NARIÑO,OSPINA,2016,99.91%,0.0%,0.36%
8,CUNDINAMARCA,LA PEÑA,2016,99.89%,0.0%,0.32%
9,NARIÑO,LA LLANADA,2018,99.89%,0.0%,3.21%


In [41]:
# Extraemos columnas de cobertura del df_raw
cobertura = df_raw[['anio', 'departamento', 'municipio', 'cobertura_neta', 'cobertura_bruta']].copy()

# Convertimos a numérico
cobertura['cobertura_neta'] = pd.to_numeric(cobertura['cobertura_neta'], errors='coerce')
cobertura['cobertura_bruta'] = pd.to_numeric(cobertura['cobertura_bruta'], errors='coerce')

# Unimos con claves de dimensiones
cobertura = cobertura.merge(dim_tiempo, on='anio', how='left')
cobertura = cobertura.merge(dim_geografia_ext, on=['departamento', 'municipio'], how='left')

# Seleccionamos columnas finales
cobertura = cobertura[['id_tiempo', 'id_geografia', 'cobertura_neta', 'cobertura_bruta']]

# Guardamos en SQLite
cobertura.to_sql("Cobertura_Educativa", conn, if_exists="replace", index=False)

print("✅ Tabla Cobertura_Educativa cargada.")


✅ Tabla Cobertura_Educativa cargada.


In [42]:
query = """
SELECT 
    g.departamento,
    ROUND(AVG(c.cobertura_neta), 3) || '%' AS cobertura_neta_promedio,
    ROUND(AVG(c.cobertura_bruta), 3) || '%' AS cobertura_bruta_promedio
FROM Cobertura_Educativa c
JOIN Dim_Geografia g ON c.id_geografia = g.id_geografia
WHERE c.cobertura_neta IS NOT NULL
GROUP BY g.departamento
ORDER BY cobertura_neta_promedio DESC
LIMIT 20;
"""

ejecutar_sql(query)


,departamento,cobertura_neta_promedio,cobertura_bruta_promedio
0,BOGOTÁ D.C.,95.89%,104.74%
1,QUINDIO,94.58%,109.718%
2,SUCRE,93.849%,111.073%
3,CESAR,93.807%,107.875%
4,MAGDALENA,93.284%,113.682%
5,META,90.741%,100.82%
6,TOLIMA,89.239%,99.657%
7,CASANARE,88.584%,99.551%
8,CUNDINAMARCA,88.551%,99.971%
9,"BOGOTÁ, D.C.",88.52%,94.909%


In [43]:
query = """
SELECT 
    g.departamento,
    ROUND(
        SUM(c.cobertura_neta * p.poblacion_5_16) / SUM(p.poblacion_5_16), 3
    ) || '%' AS cobertura_neta_ponderada,
    
    ROUND(
        SUM(c.cobertura_bruta * p.poblacion_5_16) / SUM(p.poblacion_5_16), 3
    ) || '%' AS cobertura_bruta_ponderada

FROM Cobertura_Educativa c
JOIN Poblacion_Escolar p 
    ON c.id_tiempo = p.id_tiempo AND c.id_geografia = p.id_geografia
JOIN Dim_Geografia g 
    ON c.id_geografia = g.id_geografia
WHERE c.cobertura_neta IS NOT NULL AND p.poblacion_5_16 > 0
GROUP BY g.departamento
ORDER BY cobertura_neta_ponderada DESC
LIMIT 20;
"""

ejecutar_sql(query)


,departamento,cobertura_neta_ponderada,cobertura_bruta_ponderada
0,CASANARE,96.987%,110.342%
1,BOGOTÁ D.C.,95.89%,104.74%
2,SUCRE,95.383%,111.756%
3,SANTANDER,94.028%,105.673%
4,MAGDALENA,93.709%,111.859%
5,BOLÍVAR,93.511%,106.625%
6,ARCHIPIÉLAGO DE SAN ANDRÉS. PROVIDENCIA Y SANT...,92.663%,99.146%
7,RISARALDA,92.527%,107.999%
8,CUNDINAMARCA,92.221%,102.122%
9,META,92.1%,103.919%


Este ejercicio se entrega en un archivo Jupyter Notebook (.ipynb) que contenga el código necesario para realizar las consultas en SQL y que previamente haya creado la bodega de datos con un modelo dimensional adecuado.